In [ ]:
import torchvision.transforms
import torch
from keypoint_detection.utils.heatmap import gaussian_heatmap, generate_keypoints_heatmap, overlay_image_with_heatmap, get_keypoints_from_heatmap
from keypoint_detection.models.detector import KeypointDetector


In [ ]:
import wandb
from pathlib import Path
from os import listdir
from os.path import isfile, join 
import torchvision
from PIL import Image

In [ ]:
## Get Model checkpoint from wandb


checkpoint_reference = 'tlips/icra-2022-workshop/model-2qand21y:v11'

# download checkpoint locally (if not already cached)
run = wandb.init(project="test-project", entity="tlips")
artifact = run.use_artifact(checkpoint_reference, type="model")
artifact_dir = artifact.download()

# 
#checkpoint = torch.load(Path(artifact_dir) / "model.ckpt")
#print(checkpoint["hyper_parameters"])
# load checkpoint
model = KeypointDetector.load_from_checkpoint(Path(artifact_dir) / "model.ckpt",map_location="cpu",backbone_type='Unet')

In [ ]:
IMG_FOLDER = Path("./images")
onlyfiles = [f for f in listdir(IMG_FOLDER / "original") if isfile(join(IMG_FOLDER / "original", f))]

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.ToTensor()
])

for file in onlyfiles:
    img = Image.open(IMG_FOLDER / "original" / file)
    img = transform(img)
    with torch.no_grad():
        heatmap = model(img.unsqueeze(0))
    overlay = overlay_image_with_heatmap(img,heatmap.cpu().squeeze(0),0.6)
    overlay.save(IMG_FOLDER / "heatmaps" / file)
